In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to C:\Users\ANIKET
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import tensorflow as tf
import numpy as np
import tflearn
import random 
import json

c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' 

curses is not supported on this machine (please install/reinstall curses for an optimal experience)



Scipy not supported!





In [3]:
with open(r'D:\chatbot\intents.json') as json_data:
    intents=json.load(json_data)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [5]:
words=[]
classes=[]
documents=[]
ignore=['?']
#loop through each sentence in the intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each and every word in the sentence 
        w=nltk.word_tokenize(pattern)
        #add words to the word list
        words.extend(w)
        documents.append((w,intent['tag']))
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
#perform stemming and lower each word as well as remove duplicates
words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))
#remove duplicates classes
classes =sorted(list(set(classes)))
print(len(documents),'documents')
print(len(classes),'classes',classes)
print(len(words),'unique stemmed words',words)


31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [7]:
#create training data
training =[]
output=[]
#create a empty array for output
output_empty=[0]*len(classes)
#create training set,bag of words for eachsentence
for doc in documents:
    #initialize baf =g of words
    bag=[]
    #list of tokenized words for the pattern
    pattern_words=doc[0]
    #stemming each word
    pattern_words=[stemmer.stem(words.lower()) for words in pattern_words]
    #create bag of words array
    for w in words:
        bag.append(1)if w in pattern_words else bag.append(0)
    #output is '1' for current tag and '0' for rest of other tags
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])
#shuffling features and turning it
random.shuffle(training)
training=np.array(training)
#creating training lists
train_x=list(training[:,0])
train_y=list(training[:,1])

In [ ]:
#creating underlying graph data
tf.reset_default_graph()
#Building neural network
net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)

#Defining model and setting up tensorboard
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')
#start trainig 
model.fit(train_x,train_y,n_epoch=5000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 18163  | total loss: 0.25163 | time: 0.013s
| Adam | epoch: 4541 | loss: 0.25163 - acc: 0.9629 -- iter: 24/31
Training Step: 18164  | total loss: 0.22961 | time: 0.017s
| Adam | epoch: 4541 | loss: 0.22961 - acc: 0.9666 -- iter: 31/31
--


In [28]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open("training_data","wb"))

In [29]:
#restoring all the dat structures
data=pickle.load(open('training_data','rb'))
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [30]:
with open(r'D:\chatbot\intents.json') as json_data:
    intents=json.load(json_data)

In [32]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\ANIKET SINGH\model.tflearn


In [34]:
# for users

In [33]:
def clean_up_sentence(sentence):
    #tokeizing the pattern
    sentence_words=nltk.word_tokenize(sentence)
    #sentence_words=[stemmer.stem(word.lower()) for word in sentenc words]
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [36]:
#return bag of words array:0 or 1 for each word in the bag that exits in
def bow(sentence,words,show_details=False):
    #tokenizing the pattern
    sentence_words=clean_up_sentence(sentence)
    #generating bag of words
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("found in bags: %s"%w)
    return (np.array(bag))

In [44]:
ERROR_THRESHOLD=0.3
def classify(sentence):
    #generate probabilities from the model
    results=model.predict([bow(sentence,words)])[0]
    #filter out predictions below a threshold
    results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    #sort by strength of probability
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
        #return tuple of intent and probability
    return return_list
def response(sentence,userID='123',show_details=False):
    results=classify(sentence)
    #if we have a classification then find the matching intent tag
    if results:
        #loop as long as there are matche to process
        while results:
            for i in intents['intents']:
                #find a tag matching the first result
                if i['tag']==results[0][0]:
                    #a random response from the intent
                    return print(random.choice(i['responses']))
            return pop(0)
        

In [45]:
classify('what are you hours of operation?')

[('hours', 0.9905883)]

In [46]:
response('what are you hours of operation?')

Our hours are 9am-9pm every day


In [47]:
response('what is menu for today?')

Our speciality for today is Chicken Tikka


In [49]:
response('do you accept credit card?')

We accept most major credit cards


In [50]:
response('where can we locate you?')

We are situated at the intersection of London Alley and Bridge Avenue


In [53]:
response('that is helpful')

Happy to help!


In [59]:
response('Bye')

Have a nice day
